In [15]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [16]:
from time import time
import math
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer as skTfidf
import cupy as cp           #use cupy array instead of numpy to speed up calculation by using GPU
import cudf as cf
from cuml.metrics.pairwise_distances import sparse_pairwise_distances
from cuml.feature_extraction.text import TfidfVectorizer as cuTfidf
from sklearn.metrics import pairwise_distances


import matplotlib.pyplot as plt
import sys

In [17]:
# path = '/home/test/Data/corpus.csv'
# path = '/Users/lsacy/data/corpus.csv'
path = './corpus.csv'
def load_frame(path_to_df=path, encoding='utf-16', filter_value = 100000):
    
    df= pd.read_csv(path_to_df, encoding=encoding, index_col='id')
    df.drop([df.columns[0]], inplace=True, axis=1)
    df.drop_duplicates(subset=['text'],inplace=True)
    
    exam_type_distribution = df.groupby(['exam_type'])['exam_type'].count()
    exam_type_distribution.sort_values(ascending=False)
    
    list_filtered = exam_type_distribution[exam_type_distribution > filter_value].index
    df_filtered = df[df['exam_type'].isin(list_filtered)]
    
#     print(f'original dataframe shape: {df.shape}')
#     print(f'df_filtered shape: {df_filtered.shape}')
    
#     print(f'numbers of exam types before: {len(set(df["exam_type"]))}')
#     print(f'types after filtering: {list_filtered}')
#     print(f'number of exam types after filtering: {len(set(df_filtered["exam_type"]))}')

    return df_filtered, list_filtered

In [18]:
# define a load function to load each exam type
def load_exam_type(df, exam_types):
    for i in exam_types:
        # print(f'exam type: {i}')
        dataframe = df[df['exam_type'] == i]    
        # print(f'number of documents: {dataframe.shape}')
        yield dataframe

In [19]:
def batch_tfidf(sparseMatrix, size = 5000):
    for idx, item in enumerate(range(0, sparseMatrix.shape[0], size)):
        batch_sparseMatrix = sparseMatrix[item:item+size,:]
        # print(f'batch shape: {batch_sparseMatrix.shape}, item: {item}')
        yield batch_sparseMatrix

In [20]:
def get_distances(upperbound, lower, batch_size=8000, filter=100000):

    df_filtered = load_frame(filter_value = filter)
    df_by_type = load_exam_type(df_filtered[0], df_filtered[1])
    results_dict = {}

    # loop through each exam type
    for i in range(0,len(df_filtered[1])):
        # print(f'exam {i}/{len(df_filtered[1])}')
        dataframe = next(df_by_type)
        df_indices = dataframe.index.to_list()

        tfidf = cuTfidf().fit_transform(dataframe['text'])
        
        batch = batch_tfidf(tfidf, size=batch_size)
        total_number_batches = math.ceil(tfidf.shape[0]/batch_size)
        # print(f'# of batches: {total_number_batches} | batch size: {batch_size}')
        
        # loop through bathes of tfidf matrix row wise
        counter = 0
        for i in range(0, total_number_batches):
            print(f'batch {i+1}/{total_number_batches}')
            
            batch_sparse = next(batch)
            
            distance_batch = sparse_pairwise_distances(batch_sparse, tfidf, metric='euclidean') # distance matrix

            sort_by_distance(distance_batch, df_indices, upperbound, results_dict, lower)
                                         
            del distance_batch
            del batch_sparse
                                         
            counter += batch_size
            print(f'results dict length: {len(results_dict)}')
            print('')
            break
            
        break


        del tfidf
        del batch
                
    return results_dict

In [2]:
def sort_by_distance(distance_batch, df_indices, upperbound, results, lower):
    #distance_batch= distance_batch[0:100] #take a sample of 100
    found = 0
    for i, row in tqdm(enumerate(distance_batch)):
        sorted_array = cp.sort(row)
        arg_sorted = cp.argsort(row)

        candidates, distances = get_candidates(sorted_array, lower, arg_sorted, upperbound)


        if len(candidates) > 1:
            found += 1

            df_candidates = [df_indices[int(i)] for i in candidates]
            original_index = df_candidates[0]
            df_candidates = df_candidates[1:]
            
            save_results(distances, results, original_index, df_candidates)
            
            del sorted_array, original_index, df_candidates, distances, arg_sorted, candidates
            
        if found ==5:
            break    
            print(f'{found} matches found')
    print(f'{found} matches found')

    return results

In [22]:
def save_results(distances, results, original_index, df_candidates):
    results[original_index] = (df_candidates, distances)
    return

In [23]:
def get_candidates(sorted_array, lower, arg_sorted, upper):
    candidates = []
    distances = []
    for i, x in enumerate(sorted_array):
        if x > upper or len(candidates)==3:
            break
        elif x < lower:
            continue
        else:
            candidates.append(arg_sorted[i])
            distances.append(float(x))
    return candidates, distances[1:]

In [24]:
upper_lower = [(0.1, 0.01), (0.2, 0.1), (0.25, 0.2), (0.3, 0.25), (0.35, 0.3), (0.4, 0.35), (0.45, 0.4), (0.5, 0.45), (0.55, 0.5), (0.6, 0.55), (0.65, 0.6), (0.7, 0.65), (0.8, 0.7), (0.9, 0.8), (1.0, 0.9),
               (1.1, 1.0), (1.2, 1.1), (1.3, 1.2), (1.4, 1.3)]

In [25]:
df, df_types = load_frame()

In [26]:
def print_results(df, search_results):
    for idx, (i, j) in enumerate(search_results.items()):
    # loop through every 10th item in j[0]
    #if idx ==0 or idx//10 == 0:
        # print(f'# of candidates: {len(test)}\n')
        original_text = df.loc[i]['text']
        print(f'original text:\n{original_text}')
        for i, k in enumerate(j[0]):

            text = df.loc[k]['text']

            print(f'\nVergleichstext mit distance: {j[1][i]} \n{text}')


            #print(dataframe.loc[k]['text'])

            print('')
    return

In [28]:
for i in upper_lower:
    print(f'Range for distance: {i}')
    samples = get_distances(upperbound = i[0], lower=i[1], batch_size=8000, filter=100000)
    print_results(df, samples)
    

Range for distance: (0.1, 0.01)
batch 1/13


8000it [00:05, 1475.80it/s]


1 matches found
results dict length: 1

original text:
CT Neurocranium vom    Klinik  ICB Ischämie  Methodik  Digitale Übersichtsradiographien Parallel zur OrbitoMeatalLinie gewinkelt ZeilenCT des Kopfes nativ in  mm Schichtdicke Bildschirmbefundung  Befund Es liegen keine Voraufnahmen zum Vergleich vor  Kein Nachweis einer frischen intrakraniellen Blutung Kein Anhalt für eine frische zerebrale Ischämie Mittelständiger Interhemisphärenspalt Normal weites symmetrisches Ventrikelsystem ohne Anhalt für einen Liquoraufstau Basale Zisternen und  Ventrikel frei einsehbar Keine Hirndruckzeichen Keine pathologischen Veränderungen der Schädelkalotte Kein Nachweis einer Raumforderung soweit nativ beurteilbar Regelrechte Anlage und freie Pneumatisation der NNH und der Mastoidzellen  Beurteilung Unauffälliges kraniales CT   

Vergleichstext mit distance: 0.08580908924341202 
CT Neurocranium vom    Klinik  ICB Ischämie  Methodik  Digitale Übersichtsradiographien Parallel zur OrbitoMeatalLinie gewin

7651it [00:05, 1500.55it/s]


5 matches found
results dict length: 5

original text:
CT Kopf vom   Klinik Fragestellung Rechtfertigende Indikation  Blutung Ischämie  Untersuchungstechnik Sequenzielle paraaxiale Schichtung durch das Neurokranium in primär  mm Schichtdicke Multiplanare Rekonstruktionen   Befund Keine entsprechende Voruntersuchung vorliegend  Keine intrakranielle Blutung Allseits abgrenzbarer MarkRindenKontrast und symmetrisch abgrenzbare Basalganglien Seitengleiche Dichte der Arteria cerebri media Normal weite und symmetrische interne und externe Liquorräume Keine Mittellinienverlagerung Basale Zisternen gut abgrenzbar und nicht eingeengt Nativ kein Hinweis auf eine Raumforderung  Soweit miterfasst frei belüftete Nasennebenhöhlen und Mastoidzellen ohne Spiegelbildung oder Schleimhautschwellung  Beurteilung Kein Nachweis eines frischen oder demarkierten Territorialinfarktes oder einer intrakraniellen Blutung   

Vergleichstext mit distance: 0.17725181579589844 
CT Kopf vom   Klinik Fragestellung Recht

...

5 matches found
results dict length: 5

original text:
CT Neurocranium vom    Klinik  Z n VPShunt Ventrikelweite  Befund  Es liegen die VA vom  zum Vergleich vor Ventrikeldrainage von frontal rechts mit der Spitze im rechten Seitenventrikel Unverändert erweitertes Ventrikelsystem mit einer Weite auf Höhe der SV VH von ca  mm VU  mm Unverändert erweiterte externe Liquorräume Kein Nachweis einer Mittellinienverlagerung basale Zisternen und vierter Ventrikel frei einsehbar Kein Nachweis einer frischen intrakraniellen Blutung kein Nachweis einer frischen Ischämie NNH und Mastoide bds frei  Beurteilung  Im Vergleich zur VU vom  ergibt sich keine wesentliche Befundänderung Unveränderte Darstellung des Hydrocephalus bei liegendem VPShunt  

Vergleichstext mit distance: 1.16820228099823 
CT Neurocranium vom    Klinik  Ventrikelweite  Methodik CCT nativ    Befund und Beurteilung Zum Vergleich liegt die VU vom  vor    Osteoplastische Trepanation rechts temporal  Im Vergleich zu den Voraufnahmen 

4it [00:00, 59.15it/s]


5 matches found
results dict length: 5

original text:
CT Neurocranium vom  CT Gesichtsschädel vom    Klinik Fragestellung Rechtfertigende Indikation  VK  Beurteilung  Voraufnahme vom  zum Vergleich vorliegend  Es ergibt sich weitgehend eine Befundkonstanz  Zustand nach Hemikraniektomie links und Ausräumung der Blutung links frontal regrediente Lufteinschlüsse und geringe Blutauflagerungen im Bereich des Resektionsgebietes   Links subdurale Drainage ex nur noch schmales subdurales Hämatoms links hemisphäriell idem Geringes SDH interhemiospheriell Kleine Kontusionsblutung rechts frontal   Geringe SAB bds   Keine sign Mittellinienverlagerung nach Hirndrucksonde mit Spitze links frontal paraventrikulär Ventrikeldrainage über eine Bohrlochtrepanation rechts frontal mit Spitze im linken Vorderhorn Kein Anhalt für einen Ventrikelaufstau Basale Zisternen entfaltet   Bekannte Schädelfrakturen   

Vergleichstext mit distance: 1.2004109621047974 
CT Kopf vom   Klinik Fragestellung Rechtfertigend

4it [00:02,  1.96it/s]

5 matches found
results dict length: 5

original text:
CT Neurocranium vom   Klinik Meningitis Vigilanzminderung Frage nach Aufstau  Technik CCT nativ CT post Kontrastmittel  BefundBeurteilung  Zustand nach osteoplastischer Kraniektomie rechts frontal Schleimhautschwellung der NNH beidseits  Schmale Hygrome ca  mm breit beidseits frontal Unverändert zur VU akzentuierte Weite der inneren Liquorräume Der  Ventrikel hat einen Durchmesser von axial ca  mm Mittelständiger Interhemisphärenspalt  Nach Kontrastmittelgabe kein meningeales Enhancement abgrenzbar  Nebenbefundlich ae kleines Falxmeningeom rechts parietal parafalzin    Achtung Aktuell befinden sich die Bilder bei Schwill Sebastian im PACS An der korrekten Archivierung wird gearbeitet  

Vergleichstext mit distance: 1.3000080585479736 
CT Kopf vom   Klinik Fragestellung Rechtfertigende Indikation  Verlaufskontrolle bei Schlaganfall im Mediastromgebiet rechts  Methodik Natives CCT MPR DLP  mGycm  Befund und Beurteilung  Zum Vergleich

In [ ]:
import pickle
filename = 'distances'
outfile = open(filename, 'wb')
pickle.dump(test, outfile)
outfile.close()

# infile = open(filename, 'rb')
# new_dict = pickle.load(infile)
# infile.close